In [1]:
from selenium import webdriver
import pandas as pd
import time
from numpy import random
import pickle
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import numpy as np


def flatten(t):
    return [item for sublist in t for item in sublist]

def chrome(path, headless=True):
    "Chrome driver"
    opt = webdriver.ChromeOptions()
    if headless:
        opt.add_argument("--headless")
    opt.add_argument('--blink-settings=imagesEnabled=false')
    opt.add_argument('--no-sandbox')
    browser = webdriver.Chrome(path, options=opt)
    browser.implicitly_wait(10)
    browser.set_page_load_timeout(20)

    return browser

In [ ]:
def scrapper(url, driver_path='C:/Users/59898/Documents/chromedriver.exe', headless=True):
    data = []
    metraje = []
    url_publicacion=[]

    driver = chrome(driver_path, headless=headless)

    for i in url:
        try:
            driver.get(i)
            time.sleep(random.uniform(2.0, 7.5))
        except Exception as e:
            print('Get method failed, check URL. ', e)

        no_avisos_xpath = '//*[@id="grillaavisos"]'
        grilla = driver.find_elements_by_xpath(no_avisos_xpath)
        texto_no_aviso = 'No se han encontrado avisos para la búsqueda seleccionada.'

        if grilla[0].text == texto_no_aviso:
            driver.quit()
            return
        else:
            pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
            data.append(pagina)
            html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

            for i in html:
                soup = BeautifulSoup(i, "html.parser")
                metraje.append(soup.span.text)
                url_publicacion.append(soup.find('a')['href'])

            flat_list = flatten(data)
            result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
            result['metraje'] = metraje
            result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
            result['url'] = url_publicacion
            driver.quit()

            return result

In [ ]:
base_url = 'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'
all_urls= [base_url.format(i) for i in range(1, 200)]

threads = 20

urls = np.array_split(all_urls, threads)

all_r = []

with ThreadPoolExecutor(threads) as executor:
    for result in executor.map(scrapper, urls):
        all_r.append(result)

In [ ]:
data = pd.concat(all_r).reset_index(drop=True)
data.head()

In [ ]:
data.tail()


In [ ]:
filename = f'data/ventas_mvdeo_{str(datetime.today().date())}.csv'
data.to_csv(filename, index=False)

## Recupera coordenadas

In [2]:
data = pd.read_csv('data/ventas_mvdeo_2022-08-03.csv')
data.head()

,desc,valor,metraje,url
0,3 Dormitorios en Aires Puros,$U 190.000,110.0,https://www.gallito.com.uy/excelente-garaje-2-...
1,3 Dormitorios en Prado,U$S 15.500,100.0,https://www.gallito.com.uy/gran-oportunidad-in...
2,3 Dormitorios en Villa Española,U$S 38.000,150.0,https://www.gallito.com.uy/casa-3-dormitorios-...
3,2 Dormitorios en La Teja,U$S 39.900,60.0,https://www.gallito.com.uy/economica-con-terre...
4,2 Dormitorios en Colon,U$S 45.000,50.0,https://www.gallito.com.uy/oficina-sosa-proxim...


In [3]:
def scrap_latlng(url, driver_path='C:/Users/59898/Documents/chromedriver.exe', headless=True):
    coordenadas = []
    
    driver = chrome(driver_path, headless=headless)
    
    for i in url:
        try:   
            driver.get(i)
            elem = driver.find_elements_by_css_selector('#ubicacion')
        except:
            try:
                time.sleep(3)
                driver.get(i)
                elem = driver.find_elements_by_css_selector('#ubicacion')
            except:
                elem = []
    
        if elem != []:
            element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
            element.click()
            ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
            soup = BeautifulSoup(ubic_html[0], "html.parser")
            src = soup.find('iframe')['src']
            latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
            if latlng == '/,/':
                coordenadas.append('Nan,Nan')
            else:
                coordenadas.append(latlng)
        else:
            coordenadas.append('Nan,Nan')

    driver.quit()

    return coordenadas

In [4]:
threads = 15

urls = np.array_split(list(data.url.values), threads)

len(urls)

15

In [5]:

all_r = []

with ThreadPoolExecutor(threads) as executor:
    for result in executor.map(scrap_latlng, urls):
        all_r.append(result)

In [6]:
coords = flatten(all_r)

### Guarda las coodenadas

In [19]:
data = pd.read_csv('data/ventas_mvdeo_2022-08-03.csv')
data.head()

,desc,valor,metraje,url
0,3 Dormitorios en Aires Puros,$U 190.000,110.0,https://www.gallito.com.uy/excelente-garaje-2-...
1,3 Dormitorios en Prado,U$S 15.500,100.0,https://www.gallito.com.uy/gran-oportunidad-in...
2,3 Dormitorios en Villa Española,U$S 38.000,150.0,https://www.gallito.com.uy/casa-3-dormitorios-...
3,2 Dormitorios en La Teja,U$S 39.900,60.0,https://www.gallito.com.uy/economica-con-terre...
4,2 Dormitorios en Colon,U$S 45.000,50.0,https://www.gallito.com.uy/oficina-sosa-proxim...


In [20]:
r = [i=='Nan,Nan' for i in coords]
sum(r)

168

In [25]:
coords_splitted = [i.split(',') for i in coords]
data[['lat', 'lng']] = coords_splitted

In [26]:
data['lat'] = data['lat'].astype(float).round(5)
data['lng'] = data['lng'].astype(float).round(5)

In [27]:
data.head(8)

,desc,valor,metraje,url,lat,lng
0,3 Dormitorios en Aires Puros,$U 190.000,110.0,https://www.gallito.com.uy/excelente-garaje-2-...,NaN,NaN
1,3 Dormitorios en Prado,U$S 15.500,100.0,https://www.gallito.com.uy/gran-oportunidad-in...,-34.85913,-56.20633
2,3 Dormitorios en Villa Española,U$S 38.000,150.0,https://www.gallito.com.uy/casa-3-dormitorios-...,-34.86785,-56.14285
3,2 Dormitorios en La Teja,U$S 39.900,60.0,https://www.gallito.com.uy/economica-con-terre...,-34.85720,-56.24105
4,2 Dormitorios en Colon,U$S 45.000,50.0,https://www.gallito.com.uy/oficina-sosa-proxim...,-34.80505,-56.23007
5,4 Dormitorios en Cerrito,U$S 45.000,97.0,https://www.gallito.com.uy/oportunidad-casa-de...,-34.85944,-56.15642
6,1 Dormitorio en Villa Española,U$S 48.000,39.0,https://www.gallito.com.uy/oficina-sosa-apto-1...,-32.87555,-56.02015
7,2 Dormitorios en La Teja,U$S 48.000,50.0,https://www.gallito.com.uy/casa-a-1-cuadra-de-...,-34.86604,-56.23782


In [28]:
# limpieza
data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

In [29]:
data['valor'] = data['valor'].astype(int)

In [30]:
data = data.loc[~((data.valor == 111111111) | (data.valor < 45000))]
data.head()

,desc,valor,metraje,url,lat,lng
0,3 Dormitorios en Aires Puros,190000,110.0,https://www.gallito.com.uy/excelente-garaje-2-...,NaN,NaN
4,2 Dormitorios en Colon,45000,50.0,https://www.gallito.com.uy/oficina-sosa-proxim...,-34.80505,-56.23007
5,4 Dormitorios en Cerrito,45000,97.0,https://www.gallito.com.uy/oportunidad-casa-de...,-34.85944,-56.15642
6,1 Dormitorio en Villa Española,48000,39.0,https://www.gallito.com.uy/oficina-sosa-apto-1...,-32.87555,-56.02015
7,2 Dormitorios en La Teja,48000,50.0,https://www.gallito.com.uy/casa-a-1-cuadra-de-...,-34.86604,-56.23782


In [31]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

c:\Users\59898\anaconda3\envs\scrap\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [32]:
data['barrio_ine'] = data['barrio']

c:\Users\59898\anaconda3\envs\scrap\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
def format_barrio(df, column):
    "Formatea strings de Barrio Gallito a Barrio INE"
    barrios_dict = {
        'Aguada' : 'Aguada',
        'Aires Puros' : 'Aires Puros',
        'Atahualpa' : 'Atahualpa',
        'B. De Carrasco' : 'Bañados de Carrasco',
        'Barrio Sur' : 'Barrio Sur',
        'Belvedere' : 'Belvedere',
        'Brazo Oriental' : 'Brazo Oriental',
        'Buceo' : 'Buceo',
        'Capurro' : 'Capurro, Bella Vista',
        'Carrasco' : 'Carrasco',
        'Carrasco Norte' : 'Carrasco Norte',
        'Casabo' : 'Casabó, Pajas Blancas',
        'Casavalle' : 'Casavalle',
        'Centro' : 'Centro',
        'Cerrito' : 'Cerrito',
        'Cerro' : 'Cerro',
        'Ciudad Vieja' : 'Ciudad Vieja',
        'Cno. Maldonado' : 'Colón Centro y Noroeste',
        'Colon' : 'Colón Sureste, Abayubá',
        'Cordon' : 'Cordón',
        'Golf' : 'Flor de Maroñas',
        'Ituzaingo' : 'Ituzaingó',
        'J. Hipodromo' : 'Jardines del Hipódromo',
        'Jacinto Vera' : 'Jacinto Vera',
        'La Blanqueada' : 'La Blanqueada',
        'La Comercial' : 'La Comercial',
        'La Figurita' : 'La Figurita',
        'La Teja' : 'La Teja',
        'Larrañaga' : 'Larrañaga',
        'Las Acacias' : 'Las Acacias',
        'Lezica' : 'Lezica, Melilla',
        'Malvin' : 'Malvín',
        'Malvin Norte' : 'Malvín Norte',
        'Manga' : 'Manga',
        'Maroñas' : 'Maroñas, Parque Guaraní',
        'Maroñas Curva' : 'Maroñas, Parque Guaraní',
        'Melilla' : 'Lezica, Melilla',
        'Nuevo Paris' : 'Mercado Modelo, Bolívar',
        'Pajas Blancas' : 'Nuevo París',
        'Palermo' : 'Palermo',
        'Parque Batlle' : 'Parque Batlle, Villa Dolores',
        'Parque Rodo' : 'Parque Rodó',
        'Paso De La Arena' : 'Paso de la Arena',
        'Peñarol' : 'Peñarol, Lavalleja',
        'Perez Castellanos' : 'Castro, P. Castellanos',
        'Piedras Blancas' : 'Piedras Blancas',
        'Pocitos' : 'Pocitos',
        'Pocitos Nuevo' : 'Pocitos',
        'Prado' : 'Prado, Nueva Savona',
        'Prado Norte' : 'Prado, Nueva Savona',
        'Puerto Buceo' : 'Buceo',
        'Punta Carretas' : 'Punta Carretas',
        'Punta Gorda' : 'Punta Gorda',
        'Punta Rieles' : 'Punta Rieles, Bella Italia',
        'Reducto' : 'Reducto',
        'Sayago' : 'Sayago',
        'Tres Cruces' : 'Tres Cruces',
        'Union' : 'Unión',
        'Villa Dolores' : 'Parque Batlle, Villa Dolores',
        'Villa Española' : 'Villa Española',
        'Villa Muñoz' : 'Villa Muñoz, Retiro'
        }

    return df[column].map(barrios_dict)

In [34]:
data['barrio_ine'] = format_barrio(data, 'barrio')

c:\Users\59898\anaconda3\envs\scrap\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
def encode_barrio(df, column):
    "Codifica barrio INE"
    barrios_dict = {
    'Ciudad Vieja':1,
    'Centro':2,
    'Barrio Sur':3,
    'Cordón':4,
    'Palermo':5,
    'Parque Rodó':6,
    'Punta Carretas':7,
    'Pocitos':8,
    'Buceo':9,
    'Parque Batlle, Villa Dolores':10,
    'Malvín':11,
    'Malvín Norte':12,
    'Punta Gorda':13,
    'Carrasco':14,
    'Carrasco Norte':15,
    'Bañados de Carrasco':16,
    'Maroñas, Parque Guaraní':17,
    'Flor de Maroñas':18,
    'Las Canteras':19,
    'Punta Rieles, Bella Italia':20,
    'Jardines del Hipódromo':21,
    'Ituzaingó':22,
    'Unión':23,
    'Villa Española':24,
    'Mercado Modelo, Bolívar':25,
    'Castro, P. Castellanos':26,
    'Cerrito':27,
    'Las Acacias':28,
    'Aires Puros':29,
    'Casavalle':30,
    'Piedras Blancas':31,
    'Manga, Toledo Chico':32,
    'Paso de las Duranas':33,
    'Peñarol, Lavalleja':34,
    'Cerro':35,
    'Casabó, Pajas Blancas':36,
    'La Paloma, Tomkinson':37,
    'La Teja':38,
    'Prado, Nueva Savona':39,
    'Capurro, Bella Vista':40,
    'Aguada':41,
    'Reducto':42,
    'Atahualpa':43,
    'Jacinto Vera':44,
    'La Figurita':45,
    'Larrañaga':46,
    'La Blanqueada':47,
    'Villa Muñoz, Retiro':48,
    'La Comercial':49,
    'Tres Cruces':50,
    'Brazo Oriental':51,
    'Sayago':52,
    'Conciliación':53,
    'Belvedere':54,
    'Nuevo París':55,
    'Tres Ombúes, Victoria':56,
    'Paso de la Arena':57,
    'Colón Sureste, Abayubá':58,
    'Colón Centro y Noroeste':59,
    'Lezica, Melilla':60,
    'Villa García, Manga Rural':61,
    'Manga':62
        }
    return df[column].map(barrios_dict)

In [36]:
data['cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')

c:\Users\59898\anaconda3\envs\scrap\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
data.to_csv('data/ventas_mvdeo_2022-08-03.csv_depurado.csv', index=False)

In [38]:
data = data.loc[(data.metraje > 10) & (data.metraje < 4000) & (~data.metraje.isna())]

data['metraje'] = data['metraje'].astype(int)

data['valor_metro'] = data['valor'] / data['metraje']

agru = data.groupby('cod_barrio_ine').median('valor_metro').reset_index()

agru['cod_barrio_ine'] = agru['cod_barrio_ine'].astype(int)
                                                       
agru.to_csv('data/datos_agrupados_2022-08-03.csv', index=False)